# Token-based Text Splitters

이 노트북에서는 LangChain의 다양한 토큰 기반 텍스트 분할기(Text Splitter)를 학습합니다.

## 학습 목표
- TokenTextSplitter를 사용한 토큰 기반 텍스트 분할
- tiktoken을 활용한 OpenAI 토크나이저 사용법
- spaCy, NLTK, KoNLPy 등 다양한 NLP 라이브러리 기반 분할기
- SentenceTransformers와 Hugging Face tokenizer 활용법

## 준비 작업

먼저 실습에 사용할 텍스트 파일을 로드합니다.

In [1]:
# data/appendix-keywords.txt 파일을 열어서 f라는 파일 객체를 생성합니다.
with open("data/appendix-keywords.txt") as f:
    file = f.read()  # 파일의 내용을 읽어서 file 변수에 저장합니다.

## 1. TokenTextSplitter with tiktoken

언어 모델에는 토큰 제한이 있습니다. 따라서 토큰 제한을 초과하지 않아야 합니다.

`TokenTextSplitter`는 텍스트를 토큰 수를 기반으로 청크를 생성할 때 유용합니다.

### tiktoken

`tiktoken`은 OpenAI에서 만든 빠른 `BPE Tokenizer`입니다.

In [2]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    # 청크 크기를 300으로 설정합니다.
    chunk_size=300,
    # 청크 간 중복되는 부분이 없도록 설정합니다.
    chunk_overlap=0,
)
# file 텍스트를 청크 단위로 분할합니다.
texts = text_splitter.split_text(file)

Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 315, which is longer than the specified 300
Created a chunk of size 305, which is longer than the specified 300
Created a chunk of size 366, which is longer than the specified 300
Created a chunk of size 330, which is longer than the specified 300
Created a chunk of size 351, which is longer than the specified 300
Created a chunk of size 378, which is longer than the specified 300
Created a chunk of size 361, which is longer than the specified 300
Created a chunk of size 350, which is longer than the specified 300
Created a chunk of size 362, which is longer than the specified 300
Created a chunk of size 335, which is longer than the specified 300
Created a chunk of size 353, which is longer than the specified 300
Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 336, which is longer than the specified 300
Created a chunk of size 324, which is longer tha

In [3]:
print(len(texts))  # 분할된 청크의 개수를 출력합니다.

51


In [4]:
# texts 리스트의 첫 번째 요소를 출력합니다.
print(texts[0])

Semantic Search


**참고**

- `CharacterTextSplitter.from_tiktoken_encoder`를 사용하는 경우, 텍스트는 `CharacterTextSplitter`에 의해서만 분할되고 `tiktoken` 토크나이저는 분할된 텍스트를 병합하는 데 사용됩니다. (이는 분할된 텍스트가 `tiktoken` 토크나이저로 측정한 청크 크기보다 클 수 있음을 의미합니다.)

- `RecursiveCharacterTextSplitter.from_tiktoken_encoder`를 사용하면 분할된 텍스트가 언어 모델에서 허용하는 토큰의 청크 크기보다 크지 않도록 할 수 있으며, 각 분할은 크기가 더 큰 경우 재귀적으로 분할됩니다. 또한 tiktoken 분할기를 직접 로드할 수 있으며, 이는 각 분할이 청크 크기보다 작음을 보장합니다.

## 2. TokenTextSplitter

`TokenTextSplitter` 클래스를 사용하여 텍스트를 토큰 단위로 분할합니다.

In [5]:
from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter(
    chunk_size=300,  # 청크 크기를 300으로 설정합니다.
    chunk_overlap=0,  # 청크 간 중복을 0으로 설정합니다.
)

# state_of_the_union 텍스트를 청크로 분할합니다.
texts = text_splitter.split_text(file)
print(texts[0])  # 분할된 텍스트의 첫 번째 청크를 출력합니다.

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝


## 3. spaCy

spaCy는 Python과 Cython 프로그래밍 언어로 작성된 고급 자연어 처리를 위한 오픈 소스 소프트웨어 라이브러리입니다.

NLTK의 또 다른 대안은 spaCy tokenizer를 사용하는 것입니다.

1. 텍스트가 분할되는 방식: **spaCy tokenizer**에 의해 분할됩니다.
2. chunk size가 측정되는 방법: **문자 수**로 측정됩니다.

In [6]:
# spaCy 설치
!pip install -qU spacy

In [7]:
# spaCy 영어 모델 다운로드
!python -m spacy download en_core_web_sm --quiet

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
import warnings
from langchain_text_splitters import SpacyTextSplitter

# 경고 메시지를 무시합니다.
warnings.filterwarnings("ignore")

# SpacyTextSplitter를 생성합니다.
text_splitter = SpacyTextSplitter(
    chunk_size=200,  # 청크 크기를 200으로 설정합니다.
    chunk_overlap=50,  # 청크 간 중복을 50으로 설정합니다.
)

In [9]:
# text_splitter를 사용하여 file 텍스트를 분할합니다.
texts = text_splitter.split_text(file)
print(texts[0])  # 분할된 텍스트의 첫 번째 요소를 출력합니다.

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된

결과를 반환하는 검색 방식입니다.


예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.


## 4. SentenceTransformers

`SentenceTransformersTokenTextSplitter`는 `sentence-transformer` 모델에 특화된 텍스트 분할기입니다.

기본 동작은 사용하고자 하는 sentence transformer 모델의 토큰 윈도우에 맞게 텍스트를 청크로 분할하는 것입니다.

In [10]:
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

# 문장 분할기를 생성하고 청크 간 중복을 0으로 설정합니다.
splitter = SentenceTransformersTokenTextSplitter(chunk_size=200, chunk_overlap=0)

In [11]:
count_start_and_stop_tokens = 2  # 시작과 종료 토큰의 개수를 2로 설정합니다.

# 텍스트의 토큰 개수에서 시작과 종료 토큰의 개수를 뺍니다.
text_token_count = splitter.count_tokens(text=file) - count_start_and_stop_tokens
print(text_token_count)  # 계산된 텍스트 토큰 개수를 출력합니다.

7686


In [12]:
text_chunks = splitter.split_text(text=file)  # 텍스트를 청크로 분할합니다.

In [13]:
# 두 번째 청크를 출력합니다.
print(text_chunks[1])  # 분할된 텍스트 청크 중 두 번째 청크를 출력합니다.

. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 [UNK] 합니다. [UNK] : " 사과 " 라는 단어를 [ 0. 65, - 0. 23, 0. 17 ] 과 [UNK] 벡터로 표현합니다. 연관키워드 : 자연어 처리, 벡터화, 딥러닝 token 정의 : 토큰은 텍스트를 더 작은 [UNK] 분할하는 [UNK] 의미합니다. 이는 일반적으로 단어, 문장, [UNK] 구절일 수 [UNK]. [UNK] : 문장 " 나는 학교에 간다 " 를 " 나는 ", " 학교에 ", " 간다 " 로 분할합니다. 연관키워드 : 토큰화, 자연어 처리, 구문 분석 tokenizer 정의 : 토크


## 5. NLTK

Natural Language Toolkit (NLTK)은 Python 프로그래밍 언어로 작성된 영어 자연어 처리(NLP)를 위한 라이브러리와 프로그램 모음입니다.

단순히 "\n\n"으로 분할하는 대신, NLTK tokenizers를 기반으로 텍스트를 분할하는 데 NLTK를 사용할 수 있습니다.

1. 텍스트 분할 방법: NLTK tokenizer에 의해 분할됩니다.
2. chunk 크기 측정 방법: 문자 수에 의해 측정됩니다.

In [14]:
# NLTK 설치
!pip install -qU nltk

In [15]:
# NLTK의 punkt 토크나이저 데이터 다운로드
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(
    chunk_size=200,  # 청크 크기를 200으로 설정합니다.
    chunk_overlap=0,  # 청크 간 중복을 0으로 설정합니다.
)

In [17]:
# text_splitter를 사용하여 file 텍스트를 분할합니다.
texts = text_splitter.split_text(file)
print(texts[0])  # 분할된 텍스트의 첫 번째 요소를 출력합니다.

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.

예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.


## 6. KoNLPy

KoNLPy(Korean NLP in Python)는 한국어 자연어 처리(NLP)를 위한 파이썬 패키지입니다.

토큰 분할은 텍스트를 토큰이라고 하는 더 작고 관리하기 쉬운 단위로 분할하는 과정을 포함합니다.

이러한 토큰은 종종 단어, 구, 기호 또는 추가 처리 및 분석에 중요한 다른 의미 있는 요소입니다.

영어와 같은 언어에서 토큰 분할은 일반적으로 공백과 구두점으로 단어를 분리하는 것을 포함합니다.

토큰 분할의 효과는 언어 구조에 대한 토크나이저의 이해에 크게 의존하며, 이는 의미 있는 토큰 생성을 보장합니다.

영어를 위해 설계된 토크나이저는 한국어와 같은 다른 언어의 고유한 의미 구조를 이해할 수 있는 능력이 없기 때문에 한국어 처리에 효과적으로 사용될 수 없습니다.

### KoNLPy의 Kkma 분석기를 사용한 한국어 토큰 분할

한국어 텍스트의 경우 KoNLPY에는 `Kkma`(Korean Knowledge Morpheme Analyzer)라는 형태소 분석기가 포함되어 있습니다.

`Kkma`는 한국어 텍스트에 대한 상세한 형태소 분석을 제공합니다.

문장을 단어로, 단어를 각각의 형태소로 분해하고 각 토큰에 대한 품사를 식별합니다.

텍스트 블록을 개별 문장으로 분할할 수 있어 긴 텍스트 처리에 특히 유용합니다.

### 사용시 고려사항

`Kkma`는 상세한 분석으로 유명하지만, 이러한 정밀성이 처리 속도에 영향을 미칠 수 있다는 점에 유의해야 합니다. 따라서 `Kkma`는 신속한 텍스트 처리보다 분석적 깊이가 우선시되는 애플리케이션에 가장 적합합니다.

In [18]:
# KoNLPy 설치
!pip install -qU konlpy

In [19]:
from langchain_text_splitters import KonlpyTextSplitter

# KonlpyTextSplitter를 사용하여 텍스트 분할기 객체를 생성합니다.
text_splitter = KonlpyTextSplitter(chunk_size=200, chunk_overlap=50)

In [20]:
texts = text_splitter.split_text(file)  # 한국어 문서를 문장 단위로 분할합니다.
print(texts[0])  # 분할된 문장 중 첫 번째 문장을 출력합니다.

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.


## 7. Hugging Face tokenizer

Hugging Face는 다양한 토크나이저를 제공합니다.

이 코드에서는 Hugging Face의 토크나이저 중 하나인 GPT2TokenizerFast를 사용하여 텍스트의 토큰 길이를 계산합니다.

텍스트 분할 방식은 다음과 같습니다:
- 전달된 문자 단위로 분할됩니다.

청크 크기 측정 방식은 다음과 같습니다:
- Hugging Face 토크나이저에 의해 계산된 토큰 수를 기준으로 합니다.

In [21]:
from transformers import GPT2TokenizerFast

# GPT-2 모델의 토크나이저를 불러옵니다.
hf_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [22]:
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    # 허깅페이스 토크나이저를 사용하여 CharacterTextSplitter 객체를 생성합니다.
    hf_tokenizer,
    chunk_size=300,
    chunk_overlap=50,
)
# state_of_the_union 텍스트를 분할하여 texts 변수에 저장합니다.
texts = text_splitter.split_text(file)

Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 315, which is longer than the specified 300
Created a chunk of size 305, which is longer than the specified 300
Created a chunk of size 366, which is longer than the specified 300
Created a chunk of size 330, which is longer than the specified 300
Created a chunk of size 351, which is longer than the specified 300
Created a chunk of size 378, which is longer than the specified 300
Created a chunk of size 361, which is longer than the specified 300
Created a chunk of size 350, which is longer than the specified 300
Created a chunk of size 362, which is longer than the specified 300
Created a chunk of size 335, which is longer than the specified 300
Created a chunk of size 353, which is longer than the specified 300
Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 336, which is longer than the specified 300
Created a chunk of size 324, which is longer tha

In [23]:
print(texts[1])  # texts 리스트의 1 번째 요소를 출력합니다.

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝


## 요약

이 노트북에서는 다양한 토큰 기반 텍스트 분할기를 학습했습니다:

1. **TokenTextSplitter with tiktoken**: OpenAI의 BPE 토크나이저를 사용한 토큰 기반 분할
2. **TokenTextSplitter**: 기본적인 토큰 단위 텍스트 분할
3. **spaCy**: 고급 NLP 라이브러리를 활용한 언어학적 분할
4. **SentenceTransformers**: sentence-transformer 모델에 특화된 분할
5. **NLTK**: 영어 자연어 처리를 위한 전통적인 분할기
6. **KoNLPy**: 한국어 형태소 분석을 통한 한국어 특화 분할
7. **Hugging Face tokenizer**: 다양한 사전 학습된 모델의 토크나이저 활용

각 분할기는 고유한 특징과 사용 사례가 있으며, 처리하려는 언어와 목적에 따라 적절한 분할기를 선택하는 것이 중요합니다.